# SynapseMNIST3D Data

In [1]:
# Install libAUC and medMNIST
!pip install libauc==1.2.0
!pip install medmnist
!pip install tensorboardX
!pip install acsconv

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 6.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fire: filename=fire-0.5.0-py2.py3-none-any.whl size=116952 sha256=06ed60516ed5547c15f4f3aeed2df79c245090e1ab37351821be404523137f1b
  Stored in directory: /root/.cache/pip/wheels/90/d4/f7/9404e5db0116bd4d43e5666eaa3e70ab53723e1e3ea40c9a95
Successfully built fire
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 1.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done

In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision.transforms as transforms
import medmnist
from medmnist import INFO, Evaluator
import os
import random
import torchvision
from torchvision import datasets, models
from torch.utils.data import DataLoader
from libauc.losses import AUCMLoss
from libauc.optimizers import PESG
from libauc.metrics import auc_roc_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from acsconv.converters import Conv3dConverter
from libauc.models import resnet18 as ResNet18
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

The ``converters`` are currently experimental. It may not support operations including (but not limited to) Functions in ``torch.nn.functional`` that involved data dimension


# Importing the dataset

In [3]:
data_flag = 'synapsemnist3d'
download = True
as_rgb = True
info = INFO[data_flag]
task = info['task']
n_channels = info['n_channels']
n_classes = len(info['label'])
DataClass = getattr(medmnist, info['python_class'])

# Parameters

In [4]:
seed = 0
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
BATCH_SIZE = 64
imratio = 0.1
total_epochs = 101 #50 75 90
#decay_epochs = [50, 75]
shape_transform = True
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Loading and Transformation

In [5]:
# Loading and Transforming the data
class Transform3D:
    def __init__(self, mul=None):
        self.mul = mul
    def __call__(self, voxel):
        if self.mul == '0.5':
            voxel = voxel * 0.5
        elif self.mul == 'random':
            voxel = voxel * np.random.uniform()
        return voxel.astype(np.float32)

train_transform = Transform3D(mul='random') if shape_transform else Transform3D()
eval_transform = Transform3D(mul='0.5') if shape_transform else Transform3D()

train_dataset = DataClass(split='train', transform=train_transform, download=download, as_rgb=as_rgb)
train_dataset_at_eval = DataClass(split='train', transform=eval_transform, download=download, as_rgb=as_rgb)
val_dataset = DataClass(split='val', transform=eval_transform, download=download, as_rgb=as_rgb)
test_dataset = DataClass(split='test', transform=eval_transform, download=download, as_rgb=as_rgb)

train_loader = data.DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
train_loader_at_eval = data.DataLoader(dataset=train_dataset_at_eval, batch_size=2*BATCH_SIZE, shuffle=False)
val_loader = data.DataLoader(dataset=val_dataset, batch_size=2*BATCH_SIZE, shuffle=False)
test_loader = data.DataLoader(dataset=test_dataset, batch_size=2*BATCH_SIZE, shuffle=False)


100%|██████████| 38034583/38034583 [00:06<00:00, 5558564.05it/s]


Using downloaded and verified file: /root/.medmnist/synapsemnist3d.npz
Using downloaded and verified file: /root/.medmnist/synapsemnist3d.npz
Using downloaded and verified file: /root/.medmnist/synapsemnist3d.npz


In [6]:
train_dataset

Dataset SynapseMNIST3D (synapsemnist3d)
    Number of datapoints: 1230
    Root location: /root/.medmnist
    Split: train
    Task: binary-class
    Number of channels: 1
    Meaning of labels: {'0': 'inhibitory synapse', '1': 'excitatory synapse'}
    Number of samples: {'train': 1230, 'val': 177, 'test': 352}
    Description: The SynapseMNIST3D is a new 3D volume dataset to classify whether a synapse is excitatory or inhibitory. It uses a 3D image volume of an adult rat acquired by a multi-beam scanning electron microscope. The original data is of the size 100×100×100um^3 and the resolution 8×8×30nm^3, where a (30um)^3 sub-volume was used in the MitoEM dataset with dense 3D mitochondria instance segmentation labels. Three neuroscience experts segment a pyramidal neuron within the whole volume and proofread all the synapses on this neuron with excitatory/inhibitory labels. For each labeled synaptic location, we crop a 3D volume of 1024×1024×1024nm^3 and resize it into 28×28×28 voxels

In [7]:
# params = {'lr': [0.001, 0.01, 0.1],
#           'momentum': [0.1, 0.5, 0.9],
#           'margin': [0.5, 1.0, 1.5],
#           'weight_decay': [1e-6, 1e-4, 1e-3]}
# scorer = make_scorer(auc_roc_score)

# Training

In [8]:
# Defining the ResNet-18 3D model
model = ResNet18(pretrained=False)
model = model.cuda()
model.conv1 = nn.Conv2d(3, 64, kernel_size=3,stride=1, padding=1, bias=False)
model = Conv3dConverter(model)

num_ftrs = model.fc.in_features
model.fc = nn.Sequential(
    nn.Linear(num_ftrs, 2)
)

# LibAuc loss function and optimizer
loss_fn = AUCMLoss()
optimizer = PESG(model, loss_fn=loss_fn, momentum=0.8, margin=1.0, epoch_decay=1e-3, lr=0.01, weight_decay=1e-4) #epoch decay=0.05, 2e-3
#optimizer = GridSearchCV(PESG(model, loss_fn=loss_fn, epoch_decay=0.05), params, scoring=scorer, cv=5)

best_val_auc = 0
best_test_auc = 0

for epoch in range(total_epochs):
    train_loss = []
    model = model.cuda()
    model.train()
    for data, targets in train_loader:
        data, targets = data.cuda(), targets.cuda()
        y_pred = model(data)
        loss = loss_fn(y_pred, targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss.append(loss.item())

    model.eval()
    # For checking overifitting
    # train_pred_list = []
    # train_true_list = []
    # for train_data, train_targets in train_loader_at_eval:
    #     train_data = train_data.cuda()
    #     train_pred = model(train_data)
    #     train_pred_list.append(train_pred.cpu().detach().numpy())
    #     train_true_list.append(train_targets.numpy())
    # train_true = np.concatenate(train_true_list)
    # train_pred = np.concatenate(train_pred_list)
    # train_auc = auc_roc_score(train_true, train_pred)
    # train_loss = np.mean(train_loss)

    val_pred_list = []
    val_true_list = []
    for val_data, val_targets in val_loader:
         val_data = val_data.cuda()
         val_pred = model(val_data)
         val_pred_list.append(val_pred.cpu().detach().numpy())
         val_true_list.append(val_targets.numpy())
    val_true = np.concatenate(val_true_list)
    val_pred = np.concatenate(val_pred_list)
    val_auc =  auc_roc_score(val_true, val_pred)

    print("epoch: %s, val_auc: %.4f, lr: %.4f"%(epoch,val_auc[0],optimizer.lr ))    

    # Retain the best optimized model so we can use that to test
    if val_auc[0] > best_val_auc:
        best_val_auc = val_auc[0]
        final_model=model


epoch: 0, val_auc: 0.4874, lr: 0.0100
epoch: 1, val_auc: 0.4973, lr: 0.0100
epoch: 2, val_auc: 0.5346, lr: 0.0100
epoch: 3, val_auc: 0.5775, lr: 0.0100
epoch: 4, val_auc: 0.6424, lr: 0.0100
epoch: 5, val_auc: 0.5266, lr: 0.0100
epoch: 6, val_auc: 0.6361, lr: 0.0100
epoch: 7, val_auc: 0.6495, lr: 0.0100
epoch: 8, val_auc: 0.6381, lr: 0.0100
epoch: 9, val_auc: 0.6208, lr: 0.0100
epoch: 10, val_auc: 0.5975, lr: 0.0100
epoch: 11, val_auc: 0.5922, lr: 0.0100
epoch: 12, val_auc: 0.6321, lr: 0.0100
epoch: 13, val_auc: 0.6289, lr: 0.0100
epoch: 14, val_auc: 0.6549, lr: 0.0100
epoch: 15, val_auc: 0.6148, lr: 0.0100
epoch: 16, val_auc: 0.6365, lr: 0.0100
epoch: 17, val_auc: 0.5988, lr: 0.0100
epoch: 18, val_auc: 0.6277, lr: 0.0100
epoch: 19, val_auc: 0.6095, lr: 0.0100
epoch: 20, val_auc: 0.6148, lr: 0.0100
epoch: 21, val_auc: 0.6673, lr: 0.0100
epoch: 22, val_auc: 0.6276, lr: 0.0100
epoch: 23, val_auc: 0.6683, lr: 0.0100
epoch: 24, val_auc: 0.6037, lr: 0.0100
epoch: 25, val_auc: 0.6198, lr: 0.0

In [9]:
# Saving the final model
state = {
    'net': final_model.state_dict(),
}

torch.save(state, 'final_model_Synapse.pth')

# Testing

In [10]:
test_pred_list = []
test_true_list = [] 
for test_data, test_targets in test_loader:
    test_data  = test_data.cuda()
    test_pred = final_model(test_data)
    test_pred_list.append(test_pred.cpu().detach().numpy())
    test_true_list.append(test_targets.numpy())
test_true = np.concatenate(test_true_list)
test_pred = np.concatenate(test_pred_list)
test_auc =  auc_roc_score(test_true, test_pred) 

test_pred_binary = (test_pred > 0.5).astype(int)
#test_accuracy = accuracy_score(test_true, test_pred_binary)   
print(test_auc[0])  


0.7459758345279541


# Evaluation

In [11]:
# As given in Demo Eval file on canvas
def evaluate(net, test_loader):
    # Testing AUC
    score_list = list()
    label_list = list()
    for tmp_data, tmp_label in test_loader:
        tmp_data, tmp_label = tmp_data.cuda(), tmp_label.cuda()      
        tmp_score = net(tmp_data).detach().clone().cpu()
        score_list.append(tmp_score)
        label_list.append(tmp_label.cpu())
    test_label = torch.cat(label_list)
    test_score = torch.cat(score_list)
                   
    test_auc = auc_roc_score(test_label, test_score)                   
    #print("Test: %.4f"%test_auc, flush=True)
    print("Test AUC: ",test_auc[0])

evaluate(final_model, test_loader)

Test AUC:  0.7459758345279541
